In [95]:
import os

import matplotlib.pyplot as plt
import torch
from box import Box
import cv2
import numpy as np
import pandas as pd
import scipy.interpolate as interpolate

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys
# sys.path.insert(0, "/home/deeplearning/Dev/Ultrasound/ContourDetection/ultrasound_ge/")
sys.path.insert(0, "C:\\Dev\\ContourDetectionGCN\\gcn_ultrasound_segmentation")

In [3]:
from models import load_model
from CVUSInference.Utilities.model_profiler import profile_model

In [45]:
resnet18 = load_model("VAEGCN", nb_classes=12, feature_vector_size=128, backbone=18, is_gpu=True, inference_model=True)
resnet18 = resnet18.cuda()
mobilenet = load_model("VAEGCN", nb_classes=12, feature_vector_size=128, backbone="mobilenet", is_gpu=True, inference_model=True)
mobilenet = mobilenet.cuda()
densenet201 = load_model("VAEGCN", nb_classes=12, feature_vector_size=128, backbone="densenet201", is_gpu=True, inference_model=True)
densenet201 = densenet201.cuda()

loading network type: VAEGCN..


Using cache found in C:\Users\212686118/.cache\torch\hub\pytorch_vision_v0.6.0


using inference model so image_decoder will not be included in forward pass
loading network type: VAEGCN..
using inference model so image_decoder will not be included in forward pass


Using cache found in C:\Users\212686118/.cache\torch\hub\pytorch_vision_v0.6.0


loading network type: VAEGCN..


...

using inference model so image_decoder will not be included in forward pass
loading network type: VAEGCN..


Using cache found in C:\Users\212686118/.cache\torch\hub\pytorch_vision_v0.6.0


using inference model so image_decoder will not be included in forward pass
loading network type: VAEGCN..


Using cache found in C:\Users\212686118/.cache\torch\hub\pytorch_vision_v0.6.0


using inference model so image_decoder will not be included in forward pass


In [10]:
batch_size = 1
shape_3_224 = (3, 224, 224)
shape_3_128 = (3, 128, 128)

## Profile with `224x224` then `128x128`

In [107]:
profiles = []
for input_shape in [shape_3_224, shape_3_128]:
    for m, model in zip(["ResNet18", "MobileNet", "DenseNet"], [resnet18, mobilenet, densenet201]):
        res = dict(model=m, input_shape=input_shape)
        res.update(profile_model(model, (batch_size, *input_shape), print_res=False, use_cuda=True))
        profiles.append(res)
prof_df = pd.DataFrame(profiles)
prof_df

,model,input_shape,parameters,GFlops,cpu time total,self cpu time total,cuda time total,self cuda time total,cuda memory
0,ResNet18,"(3, 224, 224)",13839333,3.628680,43.411ms,13.904ms,173.444ms,77.967ms,183974400
1,MobileNet,"(3, 224, 224)",5722277,0.600762,53.838ms,21.450ms,357.501ms,163.200ms,197688320
2,DenseNet,"(3, 224, 224)",22287653,8.580449,203.718ms,84.140ms,937.031ms,712.426ms,694176256
3,ResNet18,"(3, 128, 128)",13839333,1.185924,39.863ms,12.409ms,151.637ms,68.193ms,132940288
4,MobileNet,"(3, 128, 128)",5722277,0.197216,50.013ms,20.053ms,326.927ms,150.815ms,65788416
5,DenseNet,"(3, 128, 128)",22287653,2.802828,208.945ms,86.676ms,2.400s,788.239ms,243157504


# ONNX export

In [13]:
ONNX_model_dir = "./onnx_models"
if not os.path.exists(ONNX_model_dir):
    os.makedirs(ONNX_model_dir)

In [14]:
dummy_input_3_224 = torch.zeros((1, *shape_3_224)).cuda()
dummy_input_3_128 = torch.zeros((1, *shape_3_128)).cuda()

In [15]:
torch.onnx.export(resnet18,               # model being run
                  dummy_input_3_224,                         # model input (or a tuple for multiple inputs)
                  os.path.join(ONNX_model_dir, "resnet18_3x224x224_opset11.onnx"),   # where to save the model
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

torch.onnx.export(resnet18,               # model being run
                  dummy_input_3_128,                         # model input (or a tuple for multiple inputs)
                  os.path.join(ONNX_model_dir, "resnet18_3x128x128_opset11.onnx"),   # where to save the model
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

In [16]:
torch.onnx.export(mobilenet,               # model being run
                  dummy_input_3_224,                         # model input (or a tuple for multiple inputs)
                  os.path.join(ONNX_model_dir, "mobilenet_3x224x224_opset11.onnx"),   # where to save the model
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

torch.onnx.export(mobilenet,               # model being run
                  dummy_input_3_128,                         # model input (or a tuple for multiple inputs)
                  os.path.join(ONNX_model_dir, "mobilenet_3x128x128_opset11.onnx"),   # where to save the model
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

In [17]:
torch.onnx.export(densenet201,               # model being run
                  dummy_input_3_224,                         # model input (or a tuple for multiple inputs)
                  os.path.join(ONNX_model_dir, "densenet201_3x224x224_opset11.onnx"),   # where to save the model
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

torch.onnx.export(densenet201,               # model being run
                  dummy_input_3_128,                         # model input (or a tuple for multiple inputs)
                  os.path.join(ONNX_model_dir, "densenet201_3x128x128_opset11.onnx"),   # where to save the model
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

In [105]:
c = ["a", "b", "d"]
b = c.pop(-1)
b

'd'

In [103]:
b is None

True